In [91]:
# librerias
import sys
import pandas as pd
import numpy as np

import joblib

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, accuracy_score, recall_score

In [25]:
### Asisting configuration
sys.path.append("..")

In [26]:
from src.utils.utils import load_df

In [3]:
df = load_df('../tmp/df_2_feature_engineering.pkl')

In [4]:
df=df.drop(['no_prime','si_prime'],1)

# Train and test data

In [5]:
df['fraude'].value_counts()

0    26165
1      810
Name: fraude, dtype: int64

In [6]:
df_no_fraude = df[df['fraude'] == 0]
df_si_fraude = df[df['fraude'] == 1]

In [7]:
print(df_no_fraude.shape)
print(df_si_fraude.shape)

(26165, 26)
(810, 26)


In [9]:
muestra_no_fraude = df_no_fraude.sample(n=810,random_state=12)

In [13]:
muestra_si_fraude = df_si_fraude

In [14]:
df_nuevo = pd.concat([muestra_si_fraude,muestra_no_fraude],axis=0)

In [15]:
df_nuevo['fraude'].value_counts()

0    810
1    810
Name: fraude, dtype: int64

In [16]:
df_nuevo.columns

Index(['fisica', 'virtual', 'f', 'm', 'no_def', 'NA_establecimiento',
       'abarrotes', 'farmacia', 'mpago', 'restaurante', 'super', 'NA_ciudad',
       'guadalajara', 'merida', 'monterrey', 'toluca', 'monto', 'dcto',
       'cashback', 'linea_tc', 'interes_tc', 'fraude', 'sin_dia', 'cos_dia',
       'sin_hora', 'cos_hora'],
      dtype='object')

In [18]:
df_nuevo.shape

(1620, 26)

# Train test

In [120]:
X = df_nuevo.drop(columns=['fraude'], axis=1).values

In [121]:
y = df_nuevo['fraude']

In [122]:
print(X.shape,y.shape)

(1620, 25) (1620,)


In [123]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, stratify=y, random_state=12)

In [124]:
print(X.shape, X_train.shape, X_test.shape)

(1620, 25) (1296, 25) (324, 25)


# Modelo

In [125]:
model = LogisticRegression(random_state=12).fit(X_train, y_train)

In [140]:
y_pred = model.predict(X_test)

In [141]:
score = model.score(X_test, y_test)

0.45987654320987653

In [142]:
from sklearn.metrics import precision_score, accuracy_score, recall_score

In [143]:
data_train_precision = precision_score(y_test, y_pred)

In [144]:
data_train_accuracy = accuracy_score(y_test, y_pred)

In [145]:
data_train_recall = recall_score(y_test, y_pred)

In [146]:
data_train_accuracy

0.45987654320987653

In [147]:
data_train_precision

0.46285714285714286

In [148]:
data_train_recall

0.5

In [149]:
X_train_prediction = model.predict(X_train)

In [150]:
X_train_prediction

array([0, 1, 1, ..., 1, 0, 0])

# GridSearch

In [50]:
best_estimators = []

In [51]:
from sklearn.model_selection import GridSearchCV

estimator = LogisticRegression()
grid_params = {'penalty':['l2','l1'],
               'solver':['sag','saga']}
        

gs = GridSearchCV(estimator,
                  grid_params, 
                  scoring = 'precision', 
                  cv = 5, 
                  n_jobs = -1)
        
#train
gs.fit(X_train, y_train)
#best estimator
best_estimators.append(gs)

/Users/ele/.pyenv/versions/dpa_env/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [0.48865312 0.48865312        nan 0.48681581]
  category=UserWarning


In [53]:
best_estimators

[GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
              param_grid={'penalty': ['l2', 'l1'], 'solver': ['sag', 'saga']},
              scoring='precision')]

In [60]:
predicted_scores = models[0].predict_proba(X_test)

In [57]:
from sklearn.metrics import precision_score

In [80]:
punto_corte = 0.5

In [81]:
labels_model_0 = [0 if score < punto_corte else 1 for score in predicted_scores[:,1]]

In [82]:
precision_0 = precision_score(y_test, labels_model_0,average='micro')

In [83]:
precision_0

0.45987654320987653